<div style="text-align: center;">
  <img src="https://github.com/Hack-io-Data/Imagenes/blob/main/01-LogosHackio/logo_amarillo@4x.png?raw=true" alt="esquema" />
</div>


# Laboratorio ETL: Análisis del Sistema Energético en España

## Objetivo

Durante todos los laboratorios de esta semana realizarás un proceso completo de ETL para analizar la relación entre la demanda, el consumo y la generación eléctrica en diferentes provincias de España a lo largo de un año. Además, complementarán este análisis con datos demográficos y económicos extraídos del Instituto Nacional de Estadística (INE). El **objetivo principal** del análisis es **examinar cómo la demanda, el consumo y la generación eléctrica en diferentes provincias de España a lo largo de los años están influenciados por factores demográficos y económicos, como la población y el PIB provincial**. El análisis busca identificar patrones y correlaciones entre estas variables para comprender mejor las dinámicas energéticas regionales y su relación con el desarrollo socioeconómico en España.

Antes de realizar el análisis, vamos a definir las hipótesis con las que vamos a trabajar, las cuales definirán todo tu análisis y planteamiento de los laboratorios: 

- **Hipótesis 1: La demanda eléctrica está correlacionada con la población de la provincia.** Provincias con mayor población tienden a tener una mayor demanda eléctrica.
  
- **Hipótesis 2: El crecimiento económico (medido por el PIB) está correlacionado con el consumo eléctrico.** Las provincias con un PIB más alto o en crecimiento experimentan un mayor consumo de energía.

- **Hipótesis 3: La proporción de generación renovable está relacionada con factores económicos o geográficos.** Provincias con un mayor desarrollo económico o con condiciones geográficas favorables (como más horas de sol o viento) tienden a generar más energía renovable.


## Tareas Laboratorio Transformación

En este laboratorio, tu objetivo será limpiar y preparar los datos extraídos previamente de diferentes fuentes para su posterior análisis. Trabajarás con datos provenientes de la API de Red Eléctrica Española (REE) y del Instituto Nacional de Estadística (INE). Estos datos incluyen información sobre demanda y generación eléctrica a nivel provincial, así como datos demográficos y económicos. 


- Cargar los Datos Extraídos:

  - **Demanda Eléctrica:** Carga los datos de demanda eléctrica extraídos de la API de REE.

  - **Generación Eléctrica:** Carga los datos de generación eléctrica diferenciados por tipo de energía (eólica, solar, hidroeléctrica, etc.) a nivel provincial.

  - **Datos Demográficos:** Carga los datos demográficos por provincia extraídos de la web del INE.

  - **Datos Económicos:** Carga los datos del PIB por provincia obtenidos del INE.


-Limpieza de Datos:

- Datos de la API de REE:

  - **Demanda Eléctrica:**

    - **Conversión de Timestamps:** Asegúrate de que las fechas estén correctamente formateadas en `datetime`. Si es necesario, convierte los datos a un formato uniforme (por ejemplo, `YYYY-MM` para datos mensuales).

    - **Tratamiento de Valores Nulos:** Identifica y maneja los valores nulos en caso de que los haya. Puedes optar por eliminar filas con valores faltantes.

    - **Estandarización de Nombres de Provincias:** Verifica que los nombres de las provincias estén estandarizados y coincidan en todos los conjuntos de datos. Si hay inconsistencias, corrígelas.

  - **Generación Eléctrica:**

    - **Desagregación de Tecnologías:** Asegúrate de que los datos estén correctamente desglosados por tipo de energía. Revisa que los campos correspondientes a energía eólica, solar, hidroeléctrica, etc., estén bien identificados y sin errores.

    - **Normalización de Unidades:** Verifica que las unidades de energía estén estandarizadas (por ejemplo, MWh). Realiza las conversiones necesarias si se encuentran en otras unidades.

    - **Identificación de Outliers:** Revisa los valores extremos o atípicos en la generación de energía y decide si deben ser tratados o eliminados.

- Datos del INE:

  - **Datos Demográficos:**

    - **Consistencia en la Codificación de Provincias:** Asegúrate de que los nombres de las provincias en los datos demográficos coincidan con los nombres utilizados en los datos eléctricos.

    - **Revisión de Categorías:** Verifica que las categorías de edad, sexo, y nacionalidad estén correctamente etiquetadas y sean consistentes en todo el dataset.

    - **Manejo de Valores Faltantes:** Revisa la presencia de valores faltantes y decide cómo tratarlos (relleno, eliminación o sustitución).

  - **Datos Económicos:**

    - **Normalización del PIB:** Si los datos del PIB están en diferentes unidades o escalas, asegúrate de normalizarlos para que sean comparables entre provincias.

    - **Agrupación Temporal:** Si los datos económicos están disponibles en diferentes periodos temporales, agrúpalos y normalízalos para que coincidan con los datos eléctricos en términos de granularidad temporal (mensual o anual).

NOTA: Ten en cuenta que los datos los vamos a tener que insertar en una base de datos mañana, por lo que toda esta limpieza os recomendamos que la penséis para poder crear e insertar los datos mañana. 

In [2]:
# Path
import sys
sys.path.append('../')

# Data
import pandas as pd

In [102]:
df_generacion = pd.read_csv('./data/datosgeneración.csv',index_col=False)
df_demanda = pd.read_csv('./data/datosdemanda.csv',index_col=False)
df_datoseco=pd.read_csv('./data/datos_economicos.csv', encoding= "latin 1" ,sep=";")
df_datosdemo=pd.read_csv('./data/datos_demograficos.csv', encoding= "latin 1" ,sep=";")

In [103]:
df_generacion=df_generacion.drop(columns="Unnamed: 0")

In [104]:
df_generacion.dtypes

value           float64
percentage      float64
datetime         object
Tipo             object
Id_Comunidad      int64
dtype: object

In [110]:
# Intentar convertir a datetime y ver qué valores no se convierten
df_generacion['datetime'] = pd.to_datetime(df_generacion['datetime'], errors='coerce')

# Filtrar y mostrar las filas que siguen sin formato de fecha
errores_fecha = df_generacion[df_generacion['datetime'].isna()]
print("Filas con errores en la conversión de fecha:", errores_fecha)


Filas con errores en la conversión de fecha:          value  percentage datetime        Tipo  Id_Comunidad
3        7.925    0.016925      NaT  generacion          8745
4        8.683    0.016244      NaT  generacion          8745
5        8.663    0.018741      NaT  generacion          8745
6        8.167    0.018806      NaT  generacion          8745
7        8.012    0.019998      NaT  generacion          8745
..         ...         ...      ...         ...           ...
640  10694.393    0.159294      NaT  generacion            20
641  12821.849    0.146548      NaT  generacion            20
642  14016.445    0.168879      NaT  generacion            20
643   6190.632    0.105550      NaT  generacion            20
644   2813.361    0.038771      NaT  generacion            20

[377 rows x 5 columns]


In [116]:
vacios_por_columna = df_generacion.isna().sum()
vacios_por_columna

value             0
percentage        0
datetime        377
Tipo              0
Id_Comunidad      0
dtype: int64

In [111]:
df_generacion['datetime'] = pd.to_datetime(df_generacion['datetime'], errors='coerce')

In [112]:
df_generacion['datetime'].unique()

<DatetimeArray>
['2019-01-01 00:00:00+01:00', '2019-02-01 00:00:00+01:00',
 '2019-03-01 00:00:00+01:00',                       'NaT',
 '2019-11-01 00:00:00+01:00', '2019-12-01 00:00:00+01:00',
 '2020-01-01 00:00:00+01:00', '2020-02-01 00:00:00+01:00',
 '2020-03-01 00:00:00+01:00', '2020-11-01 00:00:00+01:00',
 '2020-12-01 00:00:00+01:00', '2021-01-01 00:00:00+01:00',
 '2021-02-01 00:00:00+01:00', '2021-03-01 00:00:00+01:00',
 '2021-11-01 00:00:00+01:00', '2021-12-01 00:00:00+01:00']
Length: 16, dtype: datetime64[ns, UTC+01:00]

In [113]:
df_generacion.dtypes

value                             float64
percentage                        float64
datetime        datetime64[ns, UTC+01:00]
Tipo                               object
Id_Comunidad                        int64
dtype: object

In [114]:
# # Guardar los DataFrames normalizados
# df_datosdemo.to_csv('data/normalized_df_datosdemo.csv', index=False)
# df_datoseconomicos.to_csv('data/normalized_df_datoseconomicos.csv', index=False)
# df_demanda.to_csv('data/normalized_df_demanda.csv', index=False)
# df_generacion.to_csv('data/normalized_df_generacion.csv', index=False)